In [1]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
import torch
from scipy.sparse import csgraph
import sys
import time
import argparse
import numpy as np
import torch
import copy
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import random
from scipy import sparse
from collections import defaultdict
from dataPrepose import *
from graphConvolution import *
import pymetis
torch.cuda.set_device(1)

### Model

In [2]:
import torch.nn as nn
import torch.nn.functional as F


class GCN(nn.Module):
    def __init__(self, nfeat,  nclass, dropout):
        super(GCN, self).__init__()

        self.lr1 = nn.Linear(nfeat, nclass,bias=True)
        self.lr2 = nn.Linear(nfeat, nclass,bias=True)
        self.lr3 = nn.Linear(nfeat, nclass,bias=True)
        self.lr4 = nn.Linear(nfeat, nclass,bias=True)
        self.lr5 = nn.Linear(nfeat, nclass,bias=True)
        self.dropout = dropout

    def forward(self, x, adj1,adj2, adj3,adj4,adj5):
        x_d = F.dropout(x, self.dropout, training=self.training)
        
        x1 = torch.mm(adj1,x_d)
        x1 = self.lr1(x1)
        
        x2 = torch.mm(adj2,x_d)
        x2 = self.lr2(x2)
        
        x3 = torch.mm(adj3,x_d)
        x3 = self.lr1(x3)
        
        x4 = torch.mm(adj4,x_d)
        x4 = self.lr4(x4)
        
        x5 = torch.mm(adj5,x_d)
        x5 = self.lr5(x5)
        
        x = x1+ x2 +x3+ x4+x5 
        return x

In [3]:
def train(epoch, model,record,features,adj1,adj2, adj3,adj4,adj5):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj1,adj2, adj3,adj4,adj5)
    loss_train = F.cross_entropy(output[idx_train], labels[idx_train]) 
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    model.eval()
    output = model(features, adj1,adj2, adj3,adj4,adj5)

    loss_val = F.cross_entropy(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    
    loss_test = F.cross_entropy(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'acc_test: {:.4f}'.format(acc_test.item()),
          'time: {:.4f}s'.format(time.time() - t))
    record[acc_val.item()] = acc_test.item()

In [4]:
adj, features, labels, idx_train, idx_val, idx_test = load_data(dataset = 'citeseer')


[STEP 1]: Upload citeseer dataset.
| # of nodes : 3327
| # of edges : 4614.0
| # of features : 3703
| # of clases   : 6
| # of train set : 120
| # of val set   : 500
| # of test set  : 1000


/home/zwt/1.Codes_P2CG/dataPrepose.py:39: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


In [5]:
features = np.eye(adj.shape[0])
features = torch.FloatTensor(features)
features = features.cuda()

### Edge Split

In [6]:
G = nx.from_numpy_matrix(adj.todense())
all_edges = list(G.edges())# number of edges
random.Random(1).shuffle(all_edges)
num_edges = len(all_edges)
m = int(num_edges/2)
num_nodes = adj.shape[0] #number of nodes
A_edges = all_edges[:m]
B_edges = all_edges[m:]
A_adj = np.zeros((num_nodes,num_nodes))
B_adj = np.zeros((num_nodes,num_nodes))

In [7]:
for edge in A_edges:
    A_adj[edge[0]][edge[1]] = 1
    A_adj[edge[1]][edge[0]] = 1
for edge in B_edges:
    B_adj[edge[0]][edge[1]] = 1
    B_adj[edge[1]][edge[0]] = 1

idx_train = idx_train.cuda()
idx_test = idx_test.cuda()
idx_val = idx_val.cuda()
labels = labels.cuda()

### 1. Centralized

In [41]:
A_adj_sparse = sparse.csr_matrix(adj)
adj1 = normalize_adj(A_adj_sparse + sp.eye(A_adj.shape[0]))
adj1 = torch.FloatTensor(adj1.todense())
adj1 = adj1.cuda()
adj2 = torch.mm(adj1,adj1)
adj3 = torch.mm(adj2,adj1)
adj4 = torch.mm(adj3,adj1)
adj5 = torch.mm(adj4,adj1)

In [44]:
model1 = GCN(nfeat=adj.shape[1],
        nclass=labels.max().item() + 1,
        dropout=0.5)
model1.cuda()
optimizer = optim.Adam(model1.parameters(),
                       lr=0.02, weight_decay=5e-4)
t_total = time.time()
record = {}
for epoch in range(200):
    train(epoch,model1,record,features,adj1,adj2,adj3,adj4,adj5)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
bit_list = sorted(record.keys())
bit_list.reverse()
for key in bit_list[:10]:
    value = record[key]
    print(key,value)

Epoch: 0001 loss_train: 1.7947 acc_train: 0.1417 acc_val: 0.1320 acc_test: 0.1700 time: 0.1121s
Epoch: 0002 loss_train: 1.7400 acc_train: 0.3417 acc_val: 0.4300 acc_test: 0.4300 time: 0.1111s
Epoch: 0003 loss_train: 1.6679 acc_train: 0.6667 acc_val: 0.4720 acc_test: 0.4460 time: 0.1035s
Epoch: 0004 loss_train: 1.6012 acc_train: 0.7417 acc_val: 0.4500 acc_test: 0.4370 time: 0.1026s
Epoch: 0005 loss_train: 1.5484 acc_train: 0.7083 acc_val: 0.4800 acc_test: 0.4590 time: 0.1024s
Epoch: 0006 loss_train: 1.4741 acc_train: 0.7917 acc_val: 0.4800 acc_test: 0.4690 time: 0.1023s
Epoch: 0007 loss_train: 1.3626 acc_train: 0.9000 acc_val: 0.5420 acc_test: 0.5330 time: 0.1022s
Epoch: 0008 loss_train: 1.3151 acc_train: 0.9000 acc_val: 0.5240 acc_test: 0.5290 time: 0.1023s
Epoch: 0009 loss_train: 1.3056 acc_train: 0.8500 acc_val: 0.4640 acc_test: 0.4600 time: 0.1023s
Epoch: 0010 loss_train: 1.2466 acc_train: 0.8667 acc_val: 0.4300 acc_test: 0.4360 time: 0.1024s
Epoch: 0011 loss_train: 1.1841 acc_train

Epoch: 0087 loss_train: 0.6155 acc_train: 0.8500 acc_val: 0.4220 acc_test: 0.3920 time: 0.1023s
Epoch: 0088 loss_train: 0.6997 acc_train: 0.8250 acc_val: 0.4180 acc_test: 0.3890 time: 0.1024s
Epoch: 0089 loss_train: 0.6092 acc_train: 0.8750 acc_val: 0.4240 acc_test: 0.3930 time: 0.1023s
Epoch: 0090 loss_train: 0.6630 acc_train: 0.8750 acc_val: 0.4320 acc_test: 0.3990 time: 0.1023s
Epoch: 0091 loss_train: 0.5867 acc_train: 0.8667 acc_val: 0.4500 acc_test: 0.4070 time: 0.1023s
Epoch: 0092 loss_train: 0.5774 acc_train: 0.8833 acc_val: 0.4660 acc_test: 0.4140 time: 0.1024s
Epoch: 0093 loss_train: 0.6552 acc_train: 0.8750 acc_val: 0.4340 acc_test: 0.4350 time: 0.1021s
Epoch: 0094 loss_train: 0.5933 acc_train: 0.8917 acc_val: 0.4460 acc_test: 0.4360 time: 0.1024s
Epoch: 0095 loss_train: 0.6021 acc_train: 0.8667 acc_val: 0.4640 acc_test: 0.4520 time: 0.1023s
Epoch: 0096 loss_train: 0.6163 acc_train: 0.8750 acc_val: 0.4580 acc_test: 0.4520 time: 0.1023s
Epoch: 0097 loss_train: 0.6802 acc_train

Epoch: 0173 loss_train: 0.6111 acc_train: 0.9167 acc_val: 0.5000 acc_test: 0.5010 time: 0.1021s
Epoch: 0174 loss_train: 0.5753 acc_train: 0.8833 acc_val: 0.4420 acc_test: 0.4430 time: 0.1023s
Epoch: 0175 loss_train: 0.5628 acc_train: 0.9167 acc_val: 0.4280 acc_test: 0.4290 time: 0.1023s
Epoch: 0176 loss_train: 0.5725 acc_train: 0.8917 acc_val: 0.4280 acc_test: 0.4320 time: 0.1022s
Epoch: 0177 loss_train: 0.6489 acc_train: 0.8333 acc_val: 0.4300 acc_test: 0.4350 time: 0.1021s
Epoch: 0178 loss_train: 0.5768 acc_train: 0.8917 acc_val: 0.4320 acc_test: 0.4330 time: 0.1023s
Epoch: 0179 loss_train: 0.6441 acc_train: 0.9000 acc_val: 0.4360 acc_test: 0.4340 time: 0.1023s
Epoch: 0180 loss_train: 0.6564 acc_train: 0.8750 acc_val: 0.4380 acc_test: 0.4350 time: 0.1023s
Epoch: 0181 loss_train: 0.5061 acc_train: 0.9167 acc_val: 0.4380 acc_test: 0.4410 time: 0.1021s
Epoch: 0182 loss_train: 0.6255 acc_train: 0.8250 acc_val: 0.4780 acc_test: 0.4730 time: 0.1024s
Epoch: 0183 loss_train: 0.5421 acc_train

### 2. Local

In [35]:
A_adj_sparse = sparse.csr_matrix(A_adj)
adj1 = normalize_adj(A_adj_sparse + sp.eye(A_adj.shape[0]))
adj1 = torch.FloatTensor(adj1.todense())
adj1 = adj1.cuda()
adj2 = torch.mm(adj1,adj1)
adj3 = torch.mm(adj2,adj1)
adj4 = torch.mm(adj3,adj1)
adj5 = torch.mm(adj4,adj1)

In [40]:
model1 = GCN(nfeat=adj.shape[1],
        nclass=labels.max().item() + 1,
        dropout=0.5)
model1.cuda()
optimizer = optim.Adam(model1.parameters(),
                       lr=0.02, weight_decay=5e-4)
t_total = time.time()
record = {}
for epoch in range(200):
    train(epoch,model1,record,features,adj1,adj2,adj3,adj4,adj5)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
bit_list = sorted(record.keys())
bit_list.reverse()
for key in bit_list[:10]:
    value = record[key]
    print(key,value)

Epoch: 0001 loss_train: 1.7933 acc_train: 0.1833 acc_val: 0.2380 acc_test: 0.2450 time: 0.1105s
Epoch: 0002 loss_train: 1.2142 acc_train: 0.7500 acc_val: 0.3440 acc_test: 0.3380 time: 0.1027s
Epoch: 0003 loss_train: 0.6576 acc_train: 0.9583 acc_val: 0.3080 acc_test: 0.2580 time: 0.1023s
Epoch: 0004 loss_train: 0.3955 acc_train: 0.9583 acc_val: 0.3000 acc_test: 0.2530 time: 0.1023s
Epoch: 0005 loss_train: 0.2857 acc_train: 0.9500 acc_val: 0.2900 acc_test: 0.2600 time: 0.1022s
Epoch: 0006 loss_train: 0.2032 acc_train: 0.9583 acc_val: 0.3000 acc_test: 0.2610 time: 0.1024s
Epoch: 0007 loss_train: 0.1293 acc_train: 0.9917 acc_val: 0.2580 acc_test: 0.2810 time: 0.1023s
Epoch: 0008 loss_train: 0.1470 acc_train: 0.9833 acc_val: 0.2680 acc_test: 0.2640 time: 0.1024s
Epoch: 0009 loss_train: 0.1159 acc_train: 0.9917 acc_val: 0.2780 acc_test: 0.2580 time: 0.1022s
Epoch: 0010 loss_train: 0.1104 acc_train: 0.9750 acc_val: 0.2780 acc_test: 0.2470 time: 0.1023s
Epoch: 0011 loss_train: 0.1625 acc_train

Epoch: 0087 loss_train: 0.1536 acc_train: 0.9833 acc_val: 0.3080 acc_test: 0.2490 time: 0.1023s
Epoch: 0088 loss_train: 0.1400 acc_train: 1.0000 acc_val: 0.3100 acc_test: 0.2710 time: 0.1022s
Epoch: 0089 loss_train: 0.1509 acc_train: 0.9833 acc_val: 0.3040 acc_test: 0.2720 time: 0.1022s
Epoch: 0090 loss_train: 0.1744 acc_train: 0.9750 acc_val: 0.3220 acc_test: 0.3120 time: 0.1024s
Epoch: 0091 loss_train: 0.1788 acc_train: 0.9833 acc_val: 0.3080 acc_test: 0.3000 time: 0.1022s
Epoch: 0092 loss_train: 0.1646 acc_train: 0.9750 acc_val: 0.3000 acc_test: 0.2980 time: 0.1024s
Epoch: 0093 loss_train: 0.1405 acc_train: 1.0000 acc_val: 0.3080 acc_test: 0.2970 time: 0.1024s
Epoch: 0094 loss_train: 0.1700 acc_train: 0.9750 acc_val: 0.3240 acc_test: 0.3130 time: 0.1023s
Epoch: 0095 loss_train: 0.1187 acc_train: 1.0000 acc_val: 0.3240 acc_test: 0.2660 time: 0.1022s
Epoch: 0096 loss_train: 0.1392 acc_train: 0.9833 acc_val: 0.2960 acc_test: 0.2430 time: 0.1023s
Epoch: 0097 loss_train: 0.1416 acc_train

Epoch: 0173 loss_train: 0.1515 acc_train: 0.9917 acc_val: 0.2900 acc_test: 0.2410 time: 0.1022s
Epoch: 0174 loss_train: 0.1970 acc_train: 0.9833 acc_val: 0.2860 acc_test: 0.2390 time: 0.1022s
Epoch: 0175 loss_train: 0.1587 acc_train: 0.9917 acc_val: 0.2900 acc_test: 0.2400 time: 0.1022s
Epoch: 0176 loss_train: 0.1734 acc_train: 0.9750 acc_val: 0.2820 acc_test: 0.2390 time: 0.1023s
Epoch: 0177 loss_train: 0.1627 acc_train: 0.9750 acc_val: 0.2940 acc_test: 0.2930 time: 0.1023s
Epoch: 0178 loss_train: 0.1592 acc_train: 0.9833 acc_val: 0.3080 acc_test: 0.2910 time: 0.1022s
Epoch: 0179 loss_train: 0.1579 acc_train: 0.9917 acc_val: 0.2840 acc_test: 0.2370 time: 0.1022s
Epoch: 0180 loss_train: 0.1268 acc_train: 1.0000 acc_val: 0.2820 acc_test: 0.2380 time: 0.1022s
Epoch: 0181 loss_train: 0.1580 acc_train: 0.9917 acc_val: 0.2840 acc_test: 0.2350 time: 0.1032s
Epoch: 0182 loss_train: 0.1326 acc_train: 0.9917 acc_val: 0.2820 acc_test: 0.2340 time: 0.1024s
Epoch: 0183 loss_train: 0.1521 acc_train

### 3. Clustering-DP

In [28]:
nparts = 1000
eps = 0.6
def DP(adj):  
    g=nx.Graph(adj)
    num_nodes=g.number_of_nodes()
    g_list=list()
    for i in range(num_nodes):
        nodes = np.where(adj[i] > 0)
        nodes = nodes[0]
        nodes = nodes.tolist()
        g_list.append(nodes)
    (edgecuts,parts)=pymetis.part_graph(nparts=nparts,adjacency=g_list)
    g_dic=dict()
    for i in range(nparts):
        g_dic[i] = []
    for i in range(num_nodes):
        index = parts[i]
        g_dic[index].append(i)
    P_array = np.zeros([adj.shape[0],nparts])
    count = 0
    for i in range(adj.shape[0]):
        for key in g_dic.keys():
            if i in g_dic[key]:
                P_array[i][key] = len(g_dic[key])
        neighbors = g_list[i]
        for neighbor in neighbors:
            for key in g_dic.keys():
                if neighbor in g_dic[key] and neighbor != i:
                    P_array[i][key] += 1
    P_array_01 = copy.deepcopy(P_array)
    P_array_01[P_array_01>0] =1
    n = P_array_01.shape[0]
    m=P_array_01.shape[1]
    count = 0
    B_edges = []
    for i in range(n):
        for j in range(m):
            if P_array_01[i][j]>0:
                B_edges.append((i,j))
                count = count+1
    B_adj = P_array_01
    B_adj_dp = np.zeros((n,m))
    eps_1 =eps*math.log(n)
    eps_2 = 1
    m_new = int(len(B_edges) + np.random.laplace(0, 1/eps_2, 1))
    eps_t = math.log(2*m*n/(2*m_new)-1)
    if eps_1 < eps_t:
        theta = 1/(2*eps_1)*eps_t
    else:
        theta = 1/(2*eps_1)*math.log(2*m*n/(4*m_new)+1/2*(math.exp(eps_1)-1))
    n1=0
    for edge in B_edges:
        i = edge[0]
        j= edge[1]
        B_adj_ij = B_adj[i][j] + np.random.laplace(0, 1/eps_1, 1)
        if B_adj_ij > theta:
            B_adj_dp[i][j]=1
            n1+=1
    n0=m_new-n1


    while n0 > 0:
        i = random.randint(0,n-1)
        j = random.randint(0,m-1)
        if B_adj[i][j]!=1 and B_adj_dp[i][j] !=1:
            B_adj_dp[i][j] =1
            n0 = n0-1   
    noise = B_adj - B_adj_dp
    P_ = P_array - noise
    Q_array = np.zeros([nparts,adj.shape[0]])
    for key in g_dic.keys():
        cluster_neighbors = g_dic[key]
        cluster_len = len(cluster_neighbors)
        for neighbor in cluster_neighbors:
            Q_array[key][neighbor] = 1/cluster_len
    A_ = np.matmul(P_, Q_array)
    return A_

In [33]:
B_adj_dp = DP(B_adj+np.eye(B_adj.shape[0]))
A_adj_new = A_adj + B_adj_dp
A_adj_sparse = sparse.csr_matrix(A_adj_new)
adj1 = normalize_adj(A_adj_sparse  )
adj1 = torch.FloatTensor(adj1.todense())
adj1 = adj1.cuda()
adj2 = torch.mm(adj1,adj1)
adj3 = torch.mm(adj2,adj1)
adj4 = torch.mm(adj3,adj1)
adj5 = torch.mm(adj4,adj1)

In [34]:
model1 = GCN(nfeat=adj.shape[1],
        nclass=labels.max().item() + 1,
        dropout=0.5)
model1.cuda()
optimizer = optim.Adam(model1.parameters(),
                       lr=0.02, weight_decay=5e-4)
t_total = time.time()
record = {}
for epoch in range(200):
    train(epoch,model1,record,features,adj1,adj2,adj3,adj4,adj5)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
bit_list = sorted(record.keys())
bit_list.reverse()
for key in bit_list[:10]:
    value = record[key]
    print(key,value)

Epoch: 0001 loss_train: 1.7897 acc_train: 0.1750 acc_val: 0.1820 acc_test: 0.2340 time: 0.1123s
Epoch: 0002 loss_train: 1.7145 acc_train: 0.5250 acc_val: 0.4140 acc_test: 0.4070 time: 0.1056s
Epoch: 0003 loss_train: 1.6246 acc_train: 0.8833 acc_val: 0.4400 acc_test: 0.4170 time: 0.1033s
Epoch: 0004 loss_train: 1.5262 acc_train: 0.9417 acc_val: 0.4500 acc_test: 0.4050 time: 0.1028s
Epoch: 0005 loss_train: 1.4554 acc_train: 0.9333 acc_val: 0.4520 acc_test: 0.4150 time: 0.1023s
Epoch: 0006 loss_train: 1.3530 acc_train: 0.9583 acc_val: 0.4380 acc_test: 0.4250 time: 0.1023s
Epoch: 0007 loss_train: 1.3017 acc_train: 0.9583 acc_val: 0.4260 acc_test: 0.4340 time: 0.1022s
Epoch: 0008 loss_train: 1.1908 acc_train: 0.9917 acc_val: 0.4100 acc_test: 0.4130 time: 0.1023s
Epoch: 0009 loss_train: 1.1181 acc_train: 0.9583 acc_val: 0.4120 acc_test: 0.4130 time: 0.1022s
Epoch: 0010 loss_train: 1.0996 acc_train: 0.9667 acc_val: 0.4100 acc_test: 0.4100 time: 0.1024s
Epoch: 0011 loss_train: 1.0294 acc_train

Epoch: 0087 loss_train: 0.3400 acc_train: 1.0000 acc_val: 0.3960 acc_test: 0.4060 time: 0.1022s
Epoch: 0088 loss_train: 0.4315 acc_train: 0.9917 acc_val: 0.4120 acc_test: 0.4110 time: 0.1024s
Epoch: 0089 loss_train: 0.5261 acc_train: 0.9667 acc_val: 0.4140 acc_test: 0.4030 time: 0.1021s
Epoch: 0090 loss_train: 0.4679 acc_train: 0.9833 acc_val: 0.4140 acc_test: 0.4120 time: 0.1024s
Epoch: 0091 loss_train: 0.4673 acc_train: 0.9917 acc_val: 0.4160 acc_test: 0.4180 time: 0.1021s
Epoch: 0092 loss_train: 0.4786 acc_train: 1.0000 acc_val: 0.4080 acc_test: 0.4140 time: 0.1024s
Epoch: 0093 loss_train: 0.4513 acc_train: 0.9750 acc_val: 0.3960 acc_test: 0.4210 time: 0.1023s
Epoch: 0094 loss_train: 0.4543 acc_train: 0.9750 acc_val: 0.3980 acc_test: 0.4020 time: 0.1029s
Epoch: 0095 loss_train: 0.3946 acc_train: 0.9917 acc_val: 0.3860 acc_test: 0.4000 time: 0.1023s
Epoch: 0096 loss_train: 0.4457 acc_train: 1.0000 acc_val: 0.3900 acc_test: 0.3930 time: 0.1024s
Epoch: 0097 loss_train: 0.4602 acc_train

Epoch: 0173 loss_train: 0.4765 acc_train: 0.9750 acc_val: 0.4000 acc_test: 0.3940 time: 0.1023s
Epoch: 0174 loss_train: 0.3922 acc_train: 0.9750 acc_val: 0.3980 acc_test: 0.3770 time: 0.1023s
Epoch: 0175 loss_train: 0.4083 acc_train: 0.9917 acc_val: 0.3820 acc_test: 0.3650 time: 0.1022s
Epoch: 0176 loss_train: 0.4158 acc_train: 0.9750 acc_val: 0.3880 acc_test: 0.3580 time: 0.1024s
Epoch: 0177 loss_train: 0.4295 acc_train: 0.9583 acc_val: 0.3820 acc_test: 0.3550 time: 0.1022s
Epoch: 0178 loss_train: 0.4542 acc_train: 0.9583 acc_val: 0.3780 acc_test: 0.3570 time: 0.1023s
Epoch: 0179 loss_train: 0.4052 acc_train: 0.9583 acc_val: 0.3820 acc_test: 0.3570 time: 0.1023s
Epoch: 0180 loss_train: 0.4183 acc_train: 0.9667 acc_val: 0.3820 acc_test: 0.3640 time: 0.1023s
Epoch: 0181 loss_train: 0.3924 acc_train: 0.9833 acc_val: 0.3980 acc_test: 0.3730 time: 0.1022s
Epoch: 0182 loss_train: 0.4115 acc_train: 0.9750 acc_val: 0.4100 acc_test: 0.3780 time: 0.1026s
Epoch: 0183 loss_train: 0.3875 acc_train

### 4. P2CG

In [15]:
A1 = A_adj+np.eye(A_adj.shape[0])
A2 = B_adj+np.eye(B_adj.shape[0])
A1 = A_adj
A1_2 = A1*A1
A1_3 = A1*A1*A1
A1_4 = A1*A1*A1*A1

A2 = B_adj
A2_2 = A2*A2
A2_3 = A2*A2*A2
A2_4 = A2*A2*A2*A2
DP_A1 = DP(A1)

In [16]:
A2_5 = A2*A2*A2*A2*A2
Norm_adj = normalize(adj)
Norm_adj_2 = Norm_adj*Norm_adj
Norm_adj_3 = Norm_adj_2*Norm_adj
Norm_adj_4 = Norm_adj_3*Norm_adj

In [17]:
A_adj_new = A_adj + DP(A2)
A_adj_sparse = sparse.csr_matrix(A_adj_new)
adj1 = normalize_adj(A_adj_sparse + sp.eye(A_adj.shape[0]))
adj1 = sparse_mx_to_torch_sparse_tensor(adj1).cuda()

##### k = 2

In [18]:
theta_1 = DP_A1 - A1
B_new= DP(2*A2)
theta_2 = B_new - 2*A2
A_3 = 2*DP_A1*A2+A2_2
theta_3 = DP(A_3)-A_3

In [19]:
t1= Norm_adj+ theta_3 - theta_1*theta_2
t2 = adj + theta_2
A_adj_new2 = t1*t2
A_adj_new2[A_adj_new2<0]=0
A_adj_sparse = sparse.csr_matrix(A_adj_new2)
adj2 = normalize_adj(A_adj_sparse + sp.eye(A_adj_new2.shape[0]))
adj2 = sparse_mx_to_torch_sparse_tensor(adj2).cuda()

##### k=3

In [20]:
B_new= DP(3*A2+3*A2_2)
theta_2 = B_new - (3*A2+3*A2_2)
A_3 = 3*DP_A1*A2_2+3*(A1_2+theta_1)*A2+A2_3
theta_3 = DP(A_3)-A_3

t1= Norm_adj_2+ theta_3 - theta_1*theta_2
t2 = adj + theta_2
A_adj_new3 = t1*t2
A_adj_new3[A_adj_new3<0]=0
A_adj_sparse = sparse.csr_matrix(A_adj_new3)
adj3 = normalize_adj(A_adj_sparse + sp.eye(A_adj_new3.shape[0]))
adj3 = sparse_mx_to_torch_sparse_tensor(adj3).cuda()

##### k =4

In [21]:
B_new= DP(4*A2_3+6*A2_2+4*A2)
theta_2 = B_new - (4*A2_3+6*A2_2+4*A2)
A_3 = 4*DP_A1*A2_3+6*(A1_2+theta_1)*A2_2+4*(A1_3+theta_1)*A2+A2_4
theta_3 = DP(A_3)-A_3

t1= Norm_adj_3+ theta_3 - theta_1*theta_2
t2 = adj + theta_2
A_adj_new4 = t1*t2
A_adj_new4[A_adj_new4<0]=0
A_adj_sparse = sparse.csr_matrix(A_adj_new4)
adj4 = normalize_adj(A_adj_sparse + sp.eye(A_adj_new4.shape[0]))
adj4 = sparse_mx_to_torch_sparse_tensor(adj4).cuda()

##### k = 5

In [22]:
A1 = A_adj
A2 = B_adj
B_new= DP(5*A2_4+10*A2_3+10*A2_2+5*A2)
theta_2 = B_new - (5*A2_4+10*A2_3+10*A2_2+5*A2)
A_3 = 5*DP_A1*A2_4+10*(A1_2+theta_1)*A2_3+10*(A1_3+theta_1)*A2_2+5*(A1_4+theta_1)*A2+A2_5
theta_3 = DP(A_3)-A_3

t1= Norm_adj_4+ theta_3 - theta_1*theta_2
t2 = adj + theta_2
A_adj_new5 = t1*t2
A_adj_new5[A_adj_new5<0]=0
A_adj_sparse = sparse.csr_matrix(A_adj_new5)
adj5 = normalize_adj(A_adj_sparse + sp.eye(A_adj_new5.shape[0]))
adj5 = sparse_mx_to_torch_sparse_tensor(adj5).cuda()

In [27]:
model1 = GCN(nfeat=adj.shape[1],
        nclass=labels.max().item() + 1,
        dropout=0.5)
model1.cuda()
optimizer = optim.Adam(model1.parameters(),
                       lr=0.02, weight_decay=5e-4)
t_total = time.time()
record = {}
for epoch in range(200):
    train(epoch,model1,record,features,adj1,adj2,adj3,adj4,adj5)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
bit_list = sorted(record.keys())
bit_list.reverse()
for key in bit_list[:10]:
    value = record[key]
    print(key,value)

Epoch: 0001 loss_train: 1.7916 acc_train: 0.2167 acc_val: 0.2940 acc_test: 0.2790 time: 0.0554s
Epoch: 0002 loss_train: 1.6949 acc_train: 0.3500 acc_val: 0.3820 acc_test: 0.3820 time: 0.0560s
Epoch: 0003 loss_train: 1.5497 acc_train: 0.6917 acc_val: 0.3440 acc_test: 0.3460 time: 0.0554s
Epoch: 0004 loss_train: 1.3710 acc_train: 0.8250 acc_val: 0.3200 acc_test: 0.3290 time: 0.0558s
Epoch: 0005 loss_train: 1.3346 acc_train: 0.7750 acc_val: 0.3460 acc_test: 0.3570 time: 0.0555s
Epoch: 0006 loss_train: 1.2270 acc_train: 0.8250 acc_val: 0.3720 acc_test: 0.3990 time: 0.0557s
Epoch: 0007 loss_train: 1.1369 acc_train: 0.8917 acc_val: 0.4060 acc_test: 0.4170 time: 0.0558s
Epoch: 0008 loss_train: 0.9705 acc_train: 0.9500 acc_val: 0.3980 acc_test: 0.3900 time: 0.0554s
Epoch: 0009 loss_train: 0.9515 acc_train: 0.9167 acc_val: 0.3720 acc_test: 0.3740 time: 0.0558s
Epoch: 0010 loss_train: 0.8678 acc_train: 0.8667 acc_val: 0.3840 acc_test: 0.3780 time: 0.0556s
Epoch: 0011 loss_train: 0.7366 acc_train

Epoch: 0089 loss_train: 0.5297 acc_train: 0.9583 acc_val: 0.3540 acc_test: 0.3860 time: 0.0555s
Epoch: 0090 loss_train: 0.5697 acc_train: 0.9083 acc_val: 0.3680 acc_test: 0.3960 time: 0.0555s
Epoch: 0091 loss_train: 0.4108 acc_train: 0.9667 acc_val: 0.3900 acc_test: 0.4140 time: 0.0559s
Epoch: 0092 loss_train: 0.5048 acc_train: 0.9667 acc_val: 0.4060 acc_test: 0.4210 time: 0.0553s
Epoch: 0093 loss_train: 0.5639 acc_train: 0.9667 acc_val: 0.4040 acc_test: 0.4250 time: 0.0559s
Epoch: 0094 loss_train: 0.6481 acc_train: 0.9167 acc_val: 0.3900 acc_test: 0.4130 time: 0.0574s
Epoch: 0095 loss_train: 0.5527 acc_train: 0.9500 acc_val: 0.3800 acc_test: 0.4070 time: 0.0555s
Epoch: 0096 loss_train: 0.5255 acc_train: 0.9333 acc_val: 0.3740 acc_test: 0.4050 time: 0.0559s
Epoch: 0097 loss_train: 0.5367 acc_train: 0.9333 acc_val: 0.4100 acc_test: 0.4190 time: 0.0554s
Epoch: 0098 loss_train: 0.5179 acc_train: 0.9500 acc_val: 0.3880 acc_test: 0.3960 time: 0.0556s
Epoch: 0099 loss_train: 0.5515 acc_train

Epoch: 0177 loss_train: 0.5511 acc_train: 0.9667 acc_val: 0.3760 acc_test: 0.3910 time: 0.0556s
Epoch: 0178 loss_train: 0.6150 acc_train: 0.8833 acc_val: 0.3640 acc_test: 0.3560 time: 0.0555s
Epoch: 0179 loss_train: 0.5149 acc_train: 0.9417 acc_val: 0.3660 acc_test: 0.3460 time: 0.0558s
Epoch: 0180 loss_train: 0.5065 acc_train: 0.9583 acc_val: 0.3660 acc_test: 0.3480 time: 0.0554s
Epoch: 0181 loss_train: 0.6290 acc_train: 0.9167 acc_val: 0.3640 acc_test: 0.3460 time: 0.0558s
Epoch: 0182 loss_train: 0.5715 acc_train: 0.9083 acc_val: 0.3680 acc_test: 0.3620 time: 0.0553s
Epoch: 0183 loss_train: 0.5042 acc_train: 0.9667 acc_val: 0.3840 acc_test: 0.3920 time: 0.0555s
Epoch: 0184 loss_train: 0.5228 acc_train: 0.9667 acc_val: 0.4120 acc_test: 0.4380 time: 0.0558s
Epoch: 0185 loss_train: 0.5001 acc_train: 0.9583 acc_val: 0.4320 acc_test: 0.4490 time: 0.0554s
Epoch: 0186 loss_train: 0.5856 acc_train: 0.9250 acc_val: 0.4320 acc_test: 0.4550 time: 0.0560s
Epoch: 0187 loss_train: 0.4402 acc_train